In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 0.4.1 cuda: True


In [3]:
## Load dataset
data_path = "../../../rings_varying_radius_c2k"
Data = torch.from_numpy(np.load(data_path + '/obs.npy')).float()
NUM_DATASETS, N, D = Data.shape
K = 3 ## number of clusters
SAMPLE_SIZE = 10
NUM_HIDDEN_GLOBAL = 64
NUM_HIDDEN_LOCAL = 64
NUM_STATS = 64

MCMC_SIZE = 30
BATCH_SIZE = 20
NUM_EPOCHS = 250
LEARNING_RATE = 1e-4

CUDA = torch.cuda.is_available()
PATH = 'ep-rad-broaderNN'
DEVICE = torch.device('cuda:1')

noise_sigma = torch.ones(1) * 0.05
if CUDA:
    noise_sigma = noise_sigma.cuda().to(DEVICE)
Train_Params = (NUM_EPOCHS, NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE, CUDA, DEVICE, PATH)
Model_Params = (noise_sigma, N, K, D, MCMC_SIZE)

In [4]:
from local_enc import *
from global_oneshot import *
from global_enc import *
## if reparameterize continuous variables
Reparameterized = False
# initialization
oneshot_eta = Oneshot_eta(K, D, NUM_HIDDEN_GLOBAL, NUM_STATS, CUDA, DEVICE, Reparameterized)
enc_z = Enc_z(K, D, NUM_HIDDEN_LOCAL, CUDA, DEVICE)
enc_eta = Enc_eta(K, D, NUM_HIDDEN_GLOBAL, NUM_STATS, CUDA, DEVICE, Reparameterized)
if CUDA:
    oneshot_eta.cuda().to(DEVICE)
    enc_z.cuda().to(DEVICE)
    enc_eta.cuda().to(DEVICE)
optimizer =  torch.optim.Adam(list(oneshot_eta.parameters())+list(enc_eta.parameters())+list(enc_z.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))

models = (oneshot_eta, enc_eta, enc_z)

In [5]:
enc_z.load_state_dict(torch.load("../weights/enc-z-%s" % PATH))
enc_eta.load_state_dict(torch.load("../weights/enc-eta-%s" % PATH))
oneshot_eta.load_state_dict(torch.load("../weights/oneshot-eta-%s" % PATH))

In [7]:
from ag_ep_rad import *
# optimizer2 =  torch.optim.Adam(list(oneshot_eta.parameters())+list(enc_eta.parameters())+list(enc_z.parameters()),lr=1e-4, betas=(0.9, 0.99))
train(models, EUBO_init_eta, optimizer, Data, Model_Params, Train_Params)

epoch: 0\250 (60s),  symKL_DB_eta: 1792.563,  symKL_DB_z: 80.955,  gap: 1888.323,  loss: -14647.041,  ess: 5.380
epoch: 1\250 (60s),  symKL_DB_eta: 1843.187,  symKL_DB_z: 76.864,  gap: 1938.243,  loss: -14496.938,  ess: 5.379
epoch: 2\250 (61s),  symKL_DB_eta: 1795.789,  symKL_DB_z: 76.948,  gap: 1891.039,  loss: -14375.323,  ess: 5.380
epoch: 3\250 (60s),  symKL_DB_eta: 1838.746,  symKL_DB_z: 74.778,  gap: 1934.570,  loss: -14398.884,  ess: 5.381
epoch: 4\250 (59s),  symKL_DB_eta: 1834.709,  symKL_DB_z: 75.298,  gap: 1929.789,  loss: -14395.716,  ess: 5.382
epoch: 5\250 (61s),  symKL_DB_eta: 1766.054,  symKL_DB_z: 76.020,  gap: 1861.844,  loss: -14691.751,  ess: 5.383
epoch: 6\250 (59s),  symKL_DB_eta: 1735.384,  symKL_DB_z: 76.737,  gap: 1835.294,  loss: -14638.547,  ess: 5.381
epoch: 7\250 (59s),  symKL_DB_eta: 1771.162,  symKL_DB_z: 74.807,  gap: 1868.849,  loss: -14725.875,  ess: 5.383
epoch: 8\250 (59s),  symKL_DB_eta: 1796.454,  symKL_DB_z: 74.873,  gap: 1895.020,  loss: -14477.

KeyboardInterrupt: 

In [ ]:
# # torch.save(enc_rad.state_dict(), "../weights/enc-rad-%s" % PATH)
# torch.save(enc_z.state_dict(), "../weights/enc-z-%s" % PATH)
# torch.save(enc_eta.state_dict(), "../weights/enc-eta-%s" % PATH)
# torch.save(oneshot_eta.state_dict(), "../weights/oneshot-eta-%s" % PATH)

In [ ]:
from ag_ep_rad import *

BATCH_SIZE_TEST = 25
Train_Params_Test = (NUM_EPOCHS, NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE_TEST, CUDA, DEVICE, PATH)
Model_Params_Test = (noise_sigma, N, K, D, 10)
obs, metric_step, reused = test(models, EUBO_init_eta, Data, Model_Params_Test, Train_Params_Test)
(q_mu, _, q_z, _) = reused

In [ ]:
%time plot_samples(obs, q_mu, q_z, K, PATH)

In [ ]:
incremental_gap = symkls_test.cpu().data.numpy()[1:]
M = incremental_gap.shape[0]
overall_gap = np.zeros(M)
for m in range(M):
    overall_gap[m] = incremental_gap[:m+1].sum()

In [ ]:
fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(111)
plt.yscale("log")
ax.plot(incremental_gap, label="incremental gap")
ax.plot(overall_gap, label='overall gap')
ax.legend(fontsize=14)
ax.set_xlabel('Steps')